Import statements

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import pandas as pd
import re
from sklearn import preprocessing

Sets the defualt seed to stop randomness 

In [2]:
rng = torch.default_generator
rng.manual_seed(42)

Load tokeniser and model from hugging face (replace text to change classifier)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("muhtasham/bert-small-finetuned-wnut17-ner")
model = AutoModelForSequenceClassification.from_pretrained('muhtasham/bert-small-finetuned-wnut17-ner')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at muhtasham/bert-small-finetuned-wnut17-ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load and split dataset

In [4]:
dataset = load_dataset("liar")
training = dataset["train"]
testing = dataset["test"]
train = pd.DataFrame(training)
test = pd.DataFrame(testing)

Found cached dataset liar (C:/Users/Amy/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

Training for binary classifier

In [5]:
# Replace all labels with either 1 or 0
train["label"] = train["label"].replace(1,0)
train["label"] = train["label"].replace(2,1)
train["label"] = train["label"].replace(3,1)
train["label"] = train["label"].replace(4,0)
train["label"] = train["label"].replace(5,0)
# Same for testing data
test["label"] = test["label"].replace(1,0)
test["label"] = test["label"].replace(2,1)
test["label"] = test["label"].replace(3,1)
test["label"] = test["label"].replace(4,0)
test["label"] = test["label"].replace(5,0)
# Training data preprocessing
for i in range(len(train.index)):
    # Remove punctuation
    removed = re.sub(r'[^\w\s]', '', train["statement"][i].lower())
    train["statement"][i] = removed
train2 = train[["label"]].copy()
values = []
for i in range(len(train.index)):
    # Tokenise text
    state = tokenizer.encode(train["statement"][i] , return_tensors = "pt")
    # Run tokenised text through BERT model
    state = model(state)
    # Get values and append to list
    values.append(state.logits.detach().numpy())
train_values =  np.concatenate(values)
#Testing data preprocessing
for i in range(len(test.index)):
    # Remove Punctuation
    removed = re.sub(r'[^\w\s]', '', test["statement"][i].lower())
    test["statement"][i] = removed
test2 = test[["label"]].copy()
values = []
for i in range(len(test.index)):
    # Tokenise text
    state = tokenizer.encode(test["statement"][i] , return_tensors = "pt")
    # Run tokenised text through BERT model
    state = model(state)
    # Get values and append to list
    values.append(state.logits.detach().numpy())
test_values =  np.concatenate(values)

C:\Users\Amy\AppData\Local\Temp\ipykernel_11124\2718373583.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["statement"][i] = removed
C:\Users\Amy\AppData\Local\Temp\ipykernel_11124\2718373583.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["statement"][i] = removed


Fitting the model (binary classifier)

In [6]:
#Normalise values and change to numpy array
x_train = preprocessing.normalize(train_values)
y_train = np.array(train2["label"])
#Normalise values and change to numpy array
x_test = preprocessing.normalize(test_values)
y_test = np.array(test2["label"])
# Set up model to fit with specified paremeters (parameters will change depnding on the BERT model used)
model2 = SVC(C=1,gamma = 1,probability = True,class_weight = 'balanced')
# Fit model
model2.fit(x_train,y_train)

SVC(C=1, class_weight='balanced', gamma=1, probability=True)

Test model (binary classifier)

In [7]:
# Get predicted values given the testing statements
pred = model2.predict(x_test)
# Prints the accuracy score of the classifier on the test dataset
# Compares actual values (y_test) to predicted values (pred)
print(accuracy_score(y_test , pred))
# optional save to file
#filename = ''
#pickle.dump(model2, open(filename, 'wb'))

0.5479345284489477


Load and split dataset

In [8]:
dataset = load_dataset("liar")
training = dataset["train"]
testing = dataset["test"]
train = pd.DataFrame(training)
test = pd.DataFrame(testing)

Found cached dataset liar (C:/Users/Amy/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

Training for 6 label classifier

In [9]:
# Training data preprocessing
for i in range(len(train.index)):
    # Remove punctuation
    removed = re.sub(r'[^\w\s]', '', train["statement"][i].lower())
    train["statement"][i] = removed
train2 = train[["label"]].copy()
values = []
for i in range(len(train.index)):
    # Tokenise text
    state = tokenizer.encode(train["statement"][i] , return_tensors = "pt")
    # Run tokenised text through BERT model
    state = model(state)
    # Get values and append to list
    values.append(state.logits.detach().numpy())
train_values =  np.concatenate(values)
#Testing data preprocessing
for i in range(len(test.index)):
    # Remove punctuation
    removed = re.sub(r'[^\w\s]', '', test["statement"][i].lower())
    test["statement"][i] = removed
test2 = test[["label"]].copy()
values = []
for i in range(len(test.index)):
    # Tokenise text
    state = tokenizer.encode(test["statement"][i] , return_tensors = "pt")
    # Run tokenised text through BERT model
    state = model(state)
    # Get values and append to list
    values.append(state.logits.detach().numpy())
test_values =  np.concatenate(values)

C:\Users\Amy\AppData\Local\Temp\ipykernel_11124\2193091684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["statement"][i] = removed
C:\Users\Amy\AppData\Local\Temp\ipykernel_11124\2193091684.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["statement"][i] = removed


Fitting the model (6 label classifier)

In [12]:
#Normalise values and change to numpy array
x_train = preprocessing.normalize(train_values)
y_train = np.array(train2["label"])
#Normalise values and change to numpy array
x_test = preprocessing.normalize(test_values)
y_test = np.array(test2["label"])
# Set up model to fit with specified paremeters (parameters will change depnding on the BERT model used)
model2 = SVC(C=0.1,gamma =0.1,probability = True,class_weight = 'balanced',kernel = 'linear')
# Fit model
model2.fit(x_train,y_train)

SVC(C=0.1, class_weight='balanced', gamma=0.1, kernel='linear',
    probability=True)

Test model (6 label classifier)

In [13]:
# Get predicted values given the testing statements
pred = model2.predict(x_test)
# Prints the accuracy score of the classifier on the test dataset
# Compares actual values (y_test) to predicted values (pred)
print(accuracy_score(y_test , pred))
# optional save to file
#filename = ''
#pickle.dump(model2, open(filename, 'wb'))

0.21200311769290725
